In [122]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.signal import find_peaks, peak_widths, savgol_filter
from scipy.interpolate import interp1d
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import numpy as np
import ast

from typing import Optional


COLORS = {
    "specie_type": {
        "mistura": '#c1121f',
        "arabica": '#003049',
        "conilon": '#780000'
    },
    "beverage_type": {
        'cagour': '#780000',
         'cari': '#c1121f',
         'caria': '#454139',
         'especial': '#003049',
         'blend': '#669bbc',
         'neutro': '#780000'},
    "region":{
        'florão': '#3e5641',
        'mogiana': "#a24936",
        'não informado': '#d36135',
        'minas gerais': '#282b28',
        'sao paulo': '#83bca9',
        'rondônia': '#0d1f22',
        }
}


WAVELENGHTS = [317.998, 320.704, 323.408, 326.11 , 328.809, 331.505, 334.198,
       336.889, 339.577, 342.262, 344.944, 347.623, 350.3  , 352.973,
       355.643, 358.31 , 360.974, 363.635, 366.293, 368.947, 371.598,
       374.246, 376.89 , 379.531, 382.168, 384.802, 387.433, 390.06 ,
       392.683, 395.302, 397.918, 400.531, 403.139, 405.743, 408.344,
       410.941, 413.534, 416.123, 418.708, 421.289, 423.866, 426.439,
       429.008, 431.572, 434.133, 436.689, 439.241, 441.788, 444.332,
       446.87 , 449.405, 451.935, 454.461, 456.982, 459.498, 462.01 ,
       464.518, 467.021, 469.519, 472.012, 474.501, 476.985, 479.464,
       481.938, 484.408, 486.873, 489.332, 491.787, 494.237, 496.682,
       499.122, 501.557, 503.987, 506.412, 508.831, 511.246, 513.655,
       516.059, 518.458, 520.852, 523.24 , 525.623, 528.001, 530.373,
       532.74 , 535.102, 537.458, 539.809, 542.154, 544.494, 546.828,
       549.157, 551.48 , 553.797, 556.109, 558.416, 560.716, 563.011,
       565.301, 567.584, 569.862, 572.135, 574.401, 576.662, 578.916,
       581.165, 583.409, 585.646, 587.877, 590.103, 592.322, 594.536,
       596.744, 598.946, 601.141, 603.331, 605.515, 607.693, 609.865,
       612.03 , 614.19 , 616.343, 618.491, 620.632, 622.767, 624.896,
       627.019, 629.136, 631.247, 633.351, 635.449, 637.541, 639.627,
       641.707, 643.78 , 645.847, 647.908, 649.963, 652.011, 654.053,
       656.089, 658.118, 660.141, 662.158, 664.168, 666.172, 668.17 ,
       670.162, 672.147, 674.126, 676.098, 678.064, 680.024, 681.977,
       683.924, 685.864, 687.798, 689.726, 691.648, 693.563, 695.471,
       697.373, 699.269, 701.158, 703.041, 704.918, 706.788, 708.652,
       710.509, 712.36 , 714.205, 716.043, 717.874, 719.7  , 721.519,
       723.331, 725.137, 726.937, 728.731, 730.518, 732.298, 734.073,
       735.84 , 737.602, 739.357, 741.106, 742.848, 744.585, 746.314,
       748.038, 749.755, 751.466, 753.17 , 754.869, 756.561, 758.246,
       759.926, 761.599, 763.266, 764.927, 766.581, 768.229, 769.871,
       771.507, 773.137, 774.76 , 776.378, 777.989, 779.594, 781.193,
       782.786, 784.372, 785.953, 787.528, 789.096, 790.659, 792.215,
       793.766, 795.31 , 796.849, 798.382, 799.908, 801.429, 802.944,
       804.453, 805.956, 807.454, 808.945, 810.431, 811.911, 813.385,
       814.854, 816.316, 817.774, 819.225, 820.671, 822.111, 823.546,
       824.975, 826.398, 827.816, 829.229, 830.636, 832.037, 833.433,
       834.824, 836.209, 837.589, 838.964, 840.333, 841.697, 843.056,
       844.41 , 845.759, 847.102, 848.44 , 849.773, 851.102, 852.425,
       853.743, 855.056, 856.364, 857.668, 858.966, 860.26 , 861.548,
       862.833, 864.112, 865.387, 866.657, 867.922, 869.183, 870.439,
       871.691, 872.938, 874.181, 875.419, 876.653, 877.883, 879.108,
       880.33 , 881.547, 882.759, 883.968, 885.173, 886.373, 887.57 ,
       888.762]

DATA_PATH = 'data/espectro_novas_amostras_140425.csv'

In [123]:

df = pd.read_csv(DATA_PATH, index_col=0)

df['spectrum'] = df['spectrum'].apply(ast.literal_eval) 

In [124]:
df['spectrum'] = df['spectrum'].apply(
    lambda s: savgol_filter(s, window_length=5, polyorder=2)
)

In [118]:
# Espectro original
spectrum = df['spectrum'].iloc[-1]
x = WAVELENGHTS

In [119]:
# Parâmetros
window_lengths = np.arange(3, 52, 2)  # apenas ímpares de 3 a 51
polyorder = 2
frames = []

# Criação dos frames
for wl in window_lengths:
    if wl >= len(spectrum):
        continue  # pula se a janela for maior que o espectro
    smoothed = savgol_filter(spectrum, window_length=wl, polyorder=polyorder)
    
    frames.append(go.Frame(
        data=[
            go.Scatter(x=x, y=spectrum, name="Original", line=dict(color="blue"), opacity=0.8),
            go.Scatter(x=x, y=smoothed, name=f"Smoothed", line=dict(color="#f77f00")),
            

        ],
        name=f"wl={wl}"
    ))

# Figura base com o primeiro frame (janela mínima)
first_smoothed = savgol_filter(spectrum, window_length=window_lengths[0], polyorder=polyorder)

fig = go.Figure(
    data=[
        go.Scatter(x=x, y=spectrum, name="Original", line=dict(color="blue"), opacity=0.8),
        go.Scatter(x=x, y=first_smoothed, name="Smoothed", line=dict(color='#f77f00')),
        
        
    ],
    layout=go.Layout(
        title=f"Savitzky-Golay Smoothing: polyorder={polyorder}",
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 600, "redraw": True},
                               "fromcurrent": True}]
            }]
        }],
        sliders=[{
            "steps": [{
                "label": f"{wl}",
                "method": "animate",
                "args": [[f"wl={wl}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]
            } for wl in window_lengths if wl < len(spectrum)],
            "currentvalue": {"prefix": "Janela: "}
        }]
    ),
    frames=frames
)

fig.update_layout(
    template='plotly_white'
)

fig.show()


In [106]:
# Parâmetros de varredura
window_lengths = np.arange(5,26, 2)  # valores ímpares ≥ polyorder+2
polyorder = 2
frames = []

# Criação dos frames
for wl in window_lengths:
    if wl >= len(spectrum):
        continue  # pula se a janela for maior que o espectro

    # Derivadas suavizadas
    first_deriv = savgol_filter(spectrum, window_length=wl, polyorder=polyorder, deriv=1)
    second_deriv = savgol_filter(spectrum, window_length=wl, polyorder=polyorder, deriv=2)

    frames.append(go.Frame(
        data=[
            go.Scatter(x=x, y=first_deriv, name="1ª Derivada", line=dict(color="blue")),
            go.Scatter(x=x, y=second_deriv, name="2ª Derivada", line=dict(color="#f77f00"))
        ],
        name=f"wl={wl}"
    ))

# Frame inicial (com a menor janela)
init_first = savgol_filter(spectrum, window_length=window_lengths[0], polyorder=polyorder, deriv=1)
init_second = savgol_filter(spectrum, window_length=window_lengths[0], polyorder=polyorder, deriv=2)

# Figura base
fig = go.Figure(
    data=[
        go.Scatter(x=x, y=init_first, name="1ª Derivada", line=dict(color="blue")),
        go.Scatter(x=x, y=init_second, name="2ª Derivada", line=dict(color="#f77f00"))
    ],
    layout=go.Layout(
        title="Derivadas Suavizadas - Savitzky-Golay",
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 600, "redraw": True}, "fromcurrent": True}]
            }]
        }],
        sliders=[{
            "steps": [{
                "label": f"{wl}",
                "method": "animate",
                "args": [[f"wl={wl}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]
            } for wl in window_lengths if wl < len(spectrum)],
            "currentvalue": {"prefix": "Janela: "}
        }]
    ),
    frames=frames
)

fig.show()


### Beverage_type / Specie_type specs

In [125]:
def plot_curve_mean(df,
                    column,
                    description: str):
    

    mean_spec1, std_spec = {}, {}

    for type, specs in df.groupby(column)['spectrum']:
        stack = np.vstack(specs.values)
        mean_spec1[type] = stack.mean(axis=0)
        # Desvio padrão amostral
        std_spec[type] = stack.std(axis=0, ddof=1)
        
    n_bands = np.arange(288)

    fig = go.Figure()
    for type in mean_spec1:
        y_mean = mean_spec1[type]
        y_std = std_spec[type]
        color = COLORS[column].get(type, '#999999')
        
        def _hex_to_rgb(hex_color):
            hex_color = hex_color.lstrip("#")
            return ("rgba" + str(tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)))).replace(")", ', 0.9)')
        
        color = _hex_to_rgb(color)
        
        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean + y_std,
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ))

        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean - y_std,
            fill='tonexty',  # preenche entre esse trace e o anterior
            fillcolor=color.replace("0.9", '0.2'),
            line=dict(width=0),
            mode='lines',
            name=f"{type} ±1σ",
            hoverinfo='skip'
        ))

        # Curva da média
        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean,
            mode='lines',
            name=f"{type} (mean)",
            line=dict(color=color, width=2),
        ))

    # Layout
    fig.update_layout(
        title=description,
        xaxis_title="Índice de Banda",
        yaxis_title="Intensidade",
        template="plotly_white"
    )

    return fig

In [126]:
fig = plot_curve_mean(df,
                      'beverage_type',
                      description='Espectros Médios por Tipo de Bebida')
fig.show()

In [127]:
fig = plot_curve_mean(df,
                      'specie_type',
                      description='Espectro Médios por tipo de espécie') 
fig.show()

### Region based specs

In [128]:
def plot_curve_mean_by_region(df,
                    column,
                    description: str,
                    column_class=None):
    
    vis_data = df.copy()
    if column_class:
        vis_data = vis_data[vis_data[column] == column_class]
        
    mean_spec1, std_spec = {}, {}

    for region, specs in vis_data.groupby('region')['spectrum']:
        stack = np.vstack(specs.values)
        mean_spec1[region] = stack.mean(axis=0)
        # Desvio padrão amostral
        std_spec[region] = stack.std(axis=0, ddof=1)
        
    n_bands = np.arange(288)

    fig = go.Figure()
    for region in mean_spec1:
        y_mean = mean_spec1[region]
        y_std = std_spec[region]
        color = COLORS['region'].get(region, '#999999')
        
        def _hex_to_rgb(hex_color):
            hex_color = hex_color.lstrip("#")
            return ("rgba" + str(tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)))).replace(")", ', 0.9)')
        
        color = _hex_to_rgb(color)
        
        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean + y_std,
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ))

        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean - y_std,
            fill='tonexty',  # preenche entre esse trace e o anterior
            fillcolor=color.replace("0.9", '0.2'),
            line=dict(width=0),
            mode='lines',
            name=f"{region} ±1σ",
            hoverinfo='skip'
        ))

        # Curva da média
        fig.add_trace(go.Scatter(
            x=n_bands, y=y_mean,
            mode='lines',
            name=f"{region} (mean)",
            line=dict(color=color, width=2),
        ))

    # Layout
    fig.update_layout(
        title=description,
        xaxis_title="Índice de Banda",
        yaxis_title="Intensidade",
        template="plotly_white"
    )

    return fig

In [129]:
fig = plot_curve_mean_by_region(df,
                    'beverage_type',
                    "",
                    column_class='especial')

fig.show()

### Qualitative variables distribution

In [130]:
columns =  ['specie_type', 'region','beverage_type', 'fine class']
freq_table = df.groupby(columns).size().reset_index(name='counts')
freq_table = freq_table.sort_values('counts', ascending=False)

freq_table

,specie_type,region,beverage_type,fine class,counts
7,arabica,mogiana,especial,especial,65
0,arabica,florão,cagour,gourmet,35
12,arabica,nao informado,especial,mole,35
1,arabica,florão,cari,rio,30
15,conilon,nao informado,neutro,neutro,25
2,arabica,florão,caria,riado,25
17,mistura,nao informado,blend,blend,25
10,arabica,nao informado,caria,riado,20
5,arabica,minas gerais,cagour,dura,20
4,arabica,minas gerais,cagour,84-85,20


In [131]:
def plot_sankey(df):
    
    cols = ['specie_type', 'region', 'beverage_type', 'fine class']
    all_labels = []
    for col in cols:
        all_labels.extend(df[col].unique())
    all_labels = list(pd.unique(all_labels))  # garantir que são únicos

    label_to_index = {label: idx for idx, label in enumerate(all_labels)}
    
    source = []
    target = []
    value = []
    
    for i in range(len(cols) - 1):
        # lógica de contagem 
        group = df.groupby([cols[i], cols[i+1]]).size().reset_index(name='count')

        for _, row in group.iterrows():
            source.append(label_to_index[row[cols[i]]])
            target.append(label_to_index[row[cols[i+1]]])
            value.append(row['count'])
            
    node_in_values = np.zeros(len(all_labels), dtype=int)
    for tgt, val in zip(target, value):
        node_in_values[tgt] += val
        
    node_hovertext = [
        f"{label}<br>Número de Amostras: {val} amostras" if val > 0 else label
        for label, val in zip(all_labels, node_in_values)
    ]
            
    fig = go.Figure(data=[go.Sankey(
        arrangement='snap',
        valueformat=",.0f",
        valuesuffix=" amostras",
        
        node=dict(
            pad=35,
            thickness=30,
            line=dict(color="#2b2d42", width=0.8),
            label=all_labels,
            color="#2b2d42",
            customdata=node_hovertext,       
            hovertemplate="%{customdata}<extra></extra>"
        ),
        
        link=dict(
            source=source,
            target=target,
            value=value,
            hovertemplate=(
                        "%{source.label} – %{target.label}<br>"   
                        "Número de amostras: %{value}<extra></extra>" 
            )
        )
    )])

    fig.update_layout(
        title_text="Fluxo de especiação, bebida, qualidade e região",
        font_size=10,
        template="plotly_white",
        width=1400,    
        height=650,
        margin=dict(l=10, r=10, b=10),
        
    )

    return fig

In [132]:
fig = plot_sankey(df)
fig.show()

C:\Users\muril\AppData\Local\Temp\ipykernel_20640\1960047976.py:7: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



### Intra Sample Variability

In [133]:
def plot_sample_variability(
    df: pd.DataFrame,
    sample_size: int = 25,
    random_state: Optional[int] = None,
    color_by: Optional[str] = None,
    **filters
):
    """
    Plota espectros individuais de uma subamostra, sem média nem desvio padrão.

    Parâmetros
    ----------
    df : DataFrame
        Dados contendo a coluna 'spectrum' (lista/array) e colunas de metadados.
    sample_size : int, default 25
        Quantos espectros sortear para plotar. Usa todos se o grupo for menor.
    random_state : int ou None
        Semente do gerador de números aleatórios (reprodutibilidade).
    **filters : coluna=valor
        Filtros dinâmicos, como beverage_type='especial', region='mogiana' etc.
    """
    # 1 ─── Filtragem dinâmica
    sub = df.copy()
    for col, val in filters.items():
        if isinstance(val, list):
            sub = sub[sub[col].isin(val)]
        else:
            sub = sub[sub[col] == val]

    total = len(sub)
    if total == 0:
        raise ValueError("Nenhum registro corresponde aos filtros fornecidos.")

    # 2 ─── Sorteia amostra
    if total > sample_size:
        sub = sub.sample(n=sample_size, random_state=random_state)

    # 3 ─── Empilha espectros
    subsample = sub["spectrum"].values
    labels = sub['subtype'].values
    x = np.arange(len(subsample[0]))  # Eixo x baseado no comprimento de um espectro

    if color_by:
        labels = sub[color_by].values
        unique_labels = pd.unique(labels)
        color_palette = px.colors.qualitative.Safe 
        color_map = {label: color_palette[i % len(color_palette)] for i, label in enumerate(unique_labels)}
    else:
        labels = np.array([""] * len(subsample))
        color_map = {}
    

    # 4 ─── Plota cada espectro individualmente
    fig = go.Figure()
    
    for i in range(len(subsample)):
        label = labels[i]
        color = color_map[label] if color_by else "#1d2cf0"
        
        fig.add_trace(go.Scatter(
            x=x,
            y=subsample[i],
            mode="lines",
            line=dict(width=1, color=color),
            showlegend=(sample_size <= 15), # Mostra legenda apenas se poucos espectros
            hovertemplate=(
                f"{color_by}: {label}<br>"
            ) if color_by else (
                ''
            )
        ))

    # Layout
    title_filters = ", ".join(f"{k}={v}" for k, v in filters.items()) or "todos"
    fig.update_layout(
        title=f"Espectros Individuais ({title_filters}) — {len(sub)}/{total} exibidos",
        xaxis_title="Índice de Banda",
        yaxis_title="Intensidade",
        template="plotly_white",
        showlegend=(sample_size <= 15)
    )

    return fig, total


In [134]:
copy  = df.copy()
copy.rename(columns={'fine class': 'subtype'}, inplace=True)

fig, total = plot_sample_variability(copy,
                                     sample_size=80,
                                     random_state=42,
                                     beverage_type='cari',
                                     specie_type='arabica')
fig.show()

### X-axis Peak Drift Analysis for Beverage Type

In [135]:
### Code for finding peaks

peak_distance = 50
peak_prominence = 0.1
peak_height = 0.15

def find_most_significant_peaks(spectrum: pd.Series,
                                peaks: np.ndarray) -> list:
    
    peak_intensities = spectrum.iloc[peaks]
    sorted_peaks = peak_intensities.sort_values(ascending=False).index[:2]
    top_2_peaks = list(sorted_peaks)  
    return top_2_peaks

def first_n_second_peak(row):
    
    spectrum = pd.Series(row['spectrum'])
    
    peaks, _ = find_peaks(spectrum,
                        distance=peak_distance, 
                        prominence=peak_prominence, 
                        height=peak_height)
    
    local_peaks_idx = find_most_significant_peaks(spectrum, peaks)
    local_peaks = []
    for peak_idx in local_peaks_idx:
        local_peaks.append(pd.Series(WAVELENGHTS).iloc[peak_idx] if peak_idx > -1 else -1)
        
    local_peaks.sort()
        
    p1_position = local_peaks[0]
    p2_position = local_peaks[1]
    p1_intensity = spectrum[local_peaks_idx[0]]
    p2_intensity = spectrum[local_peaks_idx[1]]
    
    return pd.Series([p1_position, p2_position, p1_intensity, p2_intensity],index=['peak_1_position', 'peak_2_position', 'peak_1_intensity', 'peak_2_intensity'])

def groupby_violin(variavel_interesse: str,
                   df: pd.Series) -> px.violin:
    
    color_sequence = list(COLORS['beverage_type'].values())
    
    fig = px.violin(
        df,
        x='beverage_type',
        y=f'{variavel_interesse}',
        box=True,
        color='beverage_type',
        color_discrete_sequence=color_sequence,
        labels={"beverage_type": "Tipos de bebidas"}
    )

    fig.update_layout(
        xaxis_title="Beverage Types",
        yaxis_title=f"{variavel_interesse}",
        # paper_bgcolor= 'rgba(0,0,0,0)',
        # font=dict(color='black')
        template='plotly_white'
        
    )
    
    return fig

In [136]:
peaks = df.copy()
peaks[['peak_1_position', 'peak_2_position', 'peak_1_intensity', 'peak_2_intensity']] = peaks.apply(first_n_second_peak, axis=1)


In [137]:
fig = groupby_violin('peak_1_position',
                     peaks)
fig.show() 

In [138]:
fig = groupby_violin('peak_2_position',
                     peaks)
fig.show() 

### Y-axis Intensity Distribution for Beverage_type

In [139]:
fig = groupby_violin('peak_1_intensity',
                     peaks)
fig.show() 

In [140]:
fig = groupby_violin('peak_2_intensity',
                     peaks)
fig.show() 

In [141]:
from scipy.integrate import trapezoid
WAVELENGHTS = np.asarray(WAVELENGHTS)
INIT_IDX = 60
INIT_WV = WAVELENGHTS[60]

def wl2idx(target_wl: float) -> int:
    return int(np.abs(WAVELENGHTS - target_wl).argmin())

def first_peak_area(row):
    peak_wl = row['peak_1_position']
    delta = peak_wl - INIT_WV
    end_wv = peak_wl + delta
    
    end_idx = wl2idx(end_wv)
    spec = np.asarray(row['spectrum'])          

    # integração trapezoidal área do pico
    area = trapezoid(spec[INIT_IDX:end_idx+1],
                     x=WAVELENGHTS[INIT_IDX:end_idx+1])

    return pd.Series(
        [INIT_IDX, end_idx, area],
        index=['peak1_start_idx', 'peak1_end_idx', 'peak1_area']
    )


In [147]:
area = peaks.copy()
area[['peak1_start_idx', 'peak1_end_idx', 'peak1_area']] = peaks.apply(first_peak_area, axis=1)

In [148]:
area = area[area['peak1_end_idx'] != 0]

In [149]:
fig = groupby_violin('peak1_area',
                     area)
fig.show()